In [9]:
import pandas as pd
import numpy as np
from decimal import Decimal
from llms import gemini
from llms import chatGPT
from sklearn.metrics.pairwise import cosine_similarity

x_chat = chatGPT()
x_gemini = gemini()




In [10]:
df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings.csv')
# df_exp = df_exp[df_exp.Title=="Fundraising Managers"]
df_exp.Embeddings = df_exp.Embeddings.apply(lambda x: x.strip("[]").split(", "))
embeddings = []
for x in df_exp.Embeddings:
    embeddings.append([float(y) for y in x])


embeddings = np.array(embeddings)
df_exp.Embeddings = embeddings
embeddings = np.vstack(embeddings)
embeddings.shape


(11582, 3072)

In [71]:
embeddings.shape

(11582, 3072)

In [5]:
sample_df = pd.read_csv('../output/df_with_examples_embeddings.csv')
sample_df.loc[:, ["Job1_ONET", "Job2_ONET", "Job3_ONET"]] = 'N/A'

# list(sample_df.Job1_embedding.apply(ast.literal_eval))
for i in range(3):
    sample_df[f"Job{i+1}_embedding"] = sample_df[f"Job{i+1}_embedding"].apply(lambda x: [Decimal(y) for y in x.strip("[]").split(", ")])
    sample_df[f"Job{i+1}_title_embedding"] = sample_df[f"Job{i+1}_title_embedding"].apply(lambda x: [Decimal(y) for y in x.strip("[]").split(", ")])




In [20]:

# results = []
results = list(pd.read_csv("../output/onet_df.csv").values)

for index, row in list(sample_df.iterrows())[856:]:
    print("***********************")
    print(f"Processing {index}")
    name = row["organization name"]
    print(name)
    print(row["generated_description"])
        
    job_results = []
    for i in range(3): 
        example = row[f"Example{i+1}"]
        job = row[f'Job{i+1}']
        job_title = row[f'Job{i+1}_title']
        print(f"\nProcessing Example: {example} \nJob: {job}\n")
        job_embedding = np.array(row[f'Job{i+1}_embedding']).reshape(1,-1)
        job_title_embedding = np.array(row[f'Job{i+1}_title_embedding']).reshape(1,-1)
        cosine_sim = cosine_similarity(job_embedding, embeddings, 'cosine')
        
        top3 = sorted(zip(cosine_sim[0], df_exp[['Title', 'Task']].values), reverse=True)[:3]
        for cosine_sim_job, onet in top3:
            onet_title_embedding = np.array([x_chat.get_embedding(onet[0])])
            cosine_sim_title = cosine_similarity(job_title_embedding, onet_title_embedding, 'cosine')[0][0]
            print(f"{onet[0]}: {onet[1]} \nCosine Similarity: {cosine_sim_job}, Person Cosine Similarity: {cosine_sim_title}")
            results.append([name,example,row[f"situation{i+1}_conf_interval"], row[f"situation{i+1}_conf_interval_reasoning"],job,onet[0],onet[1],job_title, cosine_sim_job,cosine_sim_title])
 

    if index % 5 == 0:
        print("**Saving Results**\n\n")
        onet_df = pd.DataFrame(results, columns=["organization_name","example","situation_conf_interval", "situation_conf_interval_reasoning","job","onet_title","onet_task","example_job_title","task_similarity", "job_title_similarity"])
        onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
        onet_df.to_csv("../output/onet_df.csv", index=False)
        
    print("***********************\n\n")


***********************
Processing 856
ONEdocket
ONEdocket is an AI-powered legal technology platform that revolutionizes legal practice by automating document review and analysis. Using advanced AI algorithms, ONEdocket generates narratives, summaries, and chronologies from medical records, deposition transcripts, and employer records, streamlining the legal process and maximizing efficiency.

Processing Example: A lawyer uses ONEdocket to automatically analyze medical records from a personal injury case, generating a concise summary of the patient's medical history and injuries. This eliminates the need for manual review and saves the lawyer significant time. 
Job: Lawyers that review medical records manually and generate summaries of the information.

Medical Records Specialists: Review records for completeness, accuracy, and compliance with regulations. 
Cosine Similarity: 0.6175114727615424, Person Cosine Similarity: 0.2794341773001746
Medical Records Specialists: Transcribe medic

In [25]:
onet_df.situation_conf_interval = onet_df.situation_conf_interval.astype("float")
onet_df.to_csv("../output/onet_df.csv", index=False)

0       9
1       9
2       9
3       8
4       8
       ..
8875    7
8876    7
8877    8
8878    8
8879    8
Name: situation_conf_interval, Length: 8880, dtype: int64

In [7]:
sample_df[sample_df["organization name"]=="CodeMate AI"]

,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,...,name,Job1_embedding,Job1_title_embedding,Job2_embedding,Job2_title_embedding,Job3_embedding,Job3_title_embedding,Job1_ONET,Job2_ONET,Job3_ONET
855,22079,CodeMate AI,1-10,2024-01-08,CodeMate is an AI pair programmer that helps d...,"Artificial Intelligence (AI), Developer Tools,...","Delaware, Ohio, United States",CodeMate is an AI pair programmer that helps d...,—,—,...,CodeMate AI,"[-0.014478749595582485, -0.0067504411563277245...","[-0.02317492663860321, 0.003656588727608323, -...","[-0.011995770037174225, -0.012616804800927639,...","[-0.02317492663860321, 0.003656588727608323, -...","[-0.01958516240119934, -0.0065619973465800285,...","[-0.02317492663860321, 0.003656588727608323, -...",N/A,N/A,N/A
